In [24]:
import tools.processing as pre
import re

# use less text for now to avoid memory error
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

TIMESTEPS = 6

vocab = pre.Vocabulary(text)

# double \\n to avoid null error in tensorboard projection
text = text.replace("\n", " \\n ")

# remove extra spacing
tokens = re.sub( " +", " ", text).split(" ")[:-1]


str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the'], 'american'), (['real', 'as', 'it', 'seems', 'the', 'american'], 'dream'), (['as', 'it', 'seems', 'the', 'american', 'dream'], '\\n'), (['it', 'seems', 'the', 'american', 'dream', '\\n'], "ain't"), (['seems', 'the', 'american', 'dream', '\\n', "ain't"], 'nothing')]


In [25]:
import tools.training as tr

encoder = tr.IndexWordEncoder("Index-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.8)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

my_seed = " ".join(str_data[0]).replace("\\n", "\n")

del str_data, str_labels

In [26]:
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)



Epoch 1/20
Loss:    	 5.633865833282471
Accuracy:	 13.928858757019043
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the freeze 
 and tried money me the helpin' 
 the minnie ak the breath never run 
 and then 6 with the without all is boy in light the kill 
 the everyone winners is i you proven 
 we shit my this tyson us 
 charged i'd wake it's


Epoch 2/20
Loss:    	 5.120909690856934
Accuracy:	 17.865751266479492
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the not 
 tried to the up 
 hate motherfuckers is an a boat 
 don't a sitting to of me 
 rush and and they check the streets 
 said a frame it's for 
 posse the sorry aftermath yearin to told 
 it's loose nigga 
 as long the


Epoch 3/20
Loss:    	 4.381536483764648
Accuracy:	 23.4095401763916
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the jackass 
 niggas the police 
 the used to boldy on to the baby of 

In [38]:
decoder.temperature = 0.4

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=40)

sampler("i got the power here and there ")

------Sampling----------
seed: 
i got the power here and there 
-
result: 
i got the power here and there  
 
 you chose 
 now you got me 
 you all you got a shame 
 can all i feel the streets 
 the cop 
 got me 
 you can't punk 
 but you got a baby 

